In [1]:
import brainsss
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import AgglomerativeClustering
import scipy
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import fcluster
from scipy.cluster import hierarchy
import matplotlib as mpl
from matplotlib.pyplot import cm
import random
from scipy.stats import sem
import time
import h5py
import ants
import nibabel as nib

import matplotlib
from scipy.ndimage import gaussian_filter1d

/home/users/yandanw/.local/lib/python3.6/site-packages/ants/viz/render_surface_function.py:16: UserWarning:

Cant import Plotly. Install it `pip install chart_studio` if you want to use ants.render_surface_function



In [2]:
from time import time
import sys
import scipy
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import pandas as pd
import psutil
#from tqdm import tqdm
sys.path.insert(0, '/home/users/brezovec/.local/lib/python3.6/site-packages/lib/python/')
import ants
#import bigbadbrain as bbb
from scipy.linalg import toeplitz
import scipy.linalg as sl
from scipy.signal import convolve2d
from scipy.signal import convolve
from sklearn.linear_model import LassoLarsIC
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
%matplotlib inline

In [3]:
from scipy.interpolate import interp1d as interp1d
import seaborn as sns
import scipy.cluster.hierarchy as sch

In [4]:
func_path = '/oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset/fly_262/func_0/'

In [5]:
###########################
### PREP VISUAL STIMULI ###
###########################

vision_path = os.path.join(func_path, 'visual')
### Load Photodiode ###
t, ft_triggers, pd1, pd2 = brainsss.load_photodiode(vision_path)
stimulus_start_times = brainsss.extract_stim_times_from_pd(pd2, t)
### Unifrom the units and Make then intigers ###
stimulus_start_times = (stimulus_start_times*100).astype('int') #index needs to be an integers


####################
### Prep Fictrac ###
####################

fictrac_path = os.path.join(func_path, 'fictrac')
fictrac_raw = brainsss.load_fictrac(fictrac_path)

fps = 100
resolution = 10 #desired resolution in ms
expt_len = fictrac_raw.shape[0]/fps*1000
behaviors = ['dRotLabY', 'dRotLabZ', 'heading']
fictrac = {}
for behavior in behaviors:
    if behavior == 'dRotLabY': short = 'Y'
    elif behavior == 'dRotLabZ': short = 'Z'
    elif behavior == 'heading': short = 'h'
    fictrac[short] = brainsss.smooth_and_interp_fictrac(fictrac_raw, fps, resolution, expt_len, behavior)
    #fictrac[short] = np.roll(fictrac[short],400) # <------- misalignment is corrected !!!!!!
fictrac_timestamps = np.arange(0,expt_len,resolution)

fictrac['h'] = np.rad2deg(fictrac['h'])

def extract_traces(fictrac, stim_times, pre_window, post_window, behavior='Z'):
    traces = []
    for i in range(len(stim_times)):
        trace = fictrac[behavior][stim_times[i]-pre_window:stim_times[i]+post_window]
        if len(trace) == pre_window + post_window: # this handles fictrac that crashed or was aborted or some bullshit
            traces.append(trace)
    traces = np.asarray(traces)
    mean_trace = np.mean(traces,axis=0)
    sem_trace = scipy.stats.sem(traces,axis=0)
    return traces, mean_trace, sem_trace

loading photodiode data... done


In [6]:
heading = fictrac['h']
heading = heading%360

In [7]:
#######################
### Load Neural Data###
#######################
neural_file = os.path.join(func_path, 'functional_channel_2_moco_zscore_highpass.h5') #
with h5py.File(neural_file, 'r') as h:
    print(h['data'].shape)
    neural = h['data'][:]

(256, 128, 49, 3384)


In [9]:
##################################################################
### Downsample the 100Hz fictrac data to 3384 neual data frame ###
##################################################################
timestamps = brainsss.load_timestamps(os.path.join(func_path, 'imaging'))
fictrac_timestamps = np.arange(0,1800000,10)

def interpolate_to_neural(slice_num, timestamps):
    x = timestamps[:, slice_num]
    f = interp1d(fictrac_timestamps, heading, fill_value="extrapolate") 
    ynew = f(x)
    return ynew

fwd = fictrac['Y']
def interpolate_to_neural_fwd(slice_num, timestamps):
    x = timestamps[:, slice_num]
    f = interp1d(fictrac_timestamps, fwd, fill_value="extrapolate") 
    ynew = f(x)
    return ynew

ang = fictrac['Z']
def interpolate_to_neural_ang(slice_num, timestamps):
    x = timestamps[:, slice_num]
    f = interp1d(fictrac_timestamps, ang, fill_value="extrapolate") 
    ynew = f(x)
    return ynew

heading_interpolated = interpolate_to_neural(0,timestamps)
### redefine 0
heading_interpolated -= 180
fwd_interpolated = interpolate_to_neural_fwd(0,timestamps)
ang_interpolated = interpolate_to_neural_ang(0,timestamps)



Trying to load timestamp data from hdf5 file.
Success.


In [11]:
# Define cluster boundaries
fixation_range = (-20, 20)
away_ranges = [(-180, -160), (160, 180)]
stop_fwd_threshold = 0.5
stop_ang_threshold = 10

# Initialize empty arrays for each cluster
fixation = []
away = []
rest = []
pause = []

# Loop through each angle and categorize it
for t, angle in enumerate(heading_interpolated.flatten()):
    if angle >= fixation_range[0] and angle <= fixation_range[1]:
        if np.abs(fwd_interpolated[t])>stop_fwd_threshold or np.abs(ang_interpolated[t])>stop_ang_threshold:
            fixation.append(t)
        else:
            pause.append(t)
    elif angle >= away_ranges[0][0] and angle <= away_ranges[0][1]:
        if np.abs(fwd_interpolated[t])>stop_fwd_threshold or np.abs(ang_interpolated[t])>stop_ang_threshold:
            away.append(t)
        else:
            pause.append(t)
    elif angle >= away_ranges[1][0] and angle <= away_ranges[1][1]:
        away.append(t)
    else:
        rest.append(t)

# Print the number of angles in each cluster
print(f"Fixation: {len(fixation)}")
print(f"Away: {len(away)}")
print(f"Rest: {len(rest)}")
print(f"Pause: {len(pause)}")
# fixation, away, stop, and menotaxis are list of t (idex number in 3384)

Fixation: 342
Away: 201
Rest: 2759
Pause: 82


In [14]:
#pause = []
pre_menotaxis = []
for t in rest:
    if np.abs(fwd_interpolated[t])<stop_fwd_threshold and np.abs(ang_interpolated[t])<stop_ang_threshold:
        pause.append(t)
    else:
        pre_menotaxis.append(t)
print(f"pause: {len(pause)}")  
print(f"pre_menotaxis: {len(pre_menotaxis)}") 

pause: 1081
pre_menotaxis: 2426


In [10]:
180000/3384
# one imaging time unit is about 53 (10 ms), or ~0.5s. 
# def of sleep: 5s
# The fly needs to have no movemoent for 10 imaging time unit

53.191489361702125

In [15]:
342+201+1081+2426

4050